In [6]:
from pathlib import Path
from random import random

import cv2
import numpy as np
import rasterio
from rasterio.features import shapes
from rasterio.windows import Window
from shapely.geometry import Point, shape

In [7]:
interval = 0.62
mean_area = 0.315


def cut_polygon(geom):
    """
    Cut OBB rectangle into grids
    """
    obb = geom.minimum_rotated_rectangle.exterior.coords
    p1, p2, p3 = Point(obb[0]), Point(obb[1]), Point(obb[2])
    dist1, dist2 = p1.distance(p2), p2.distance(p3)
    if dist1 > dist2:
        cut_num = round(dist1 / interval) + int(dist1 * 2 < interval)
        short_edge = dist2
    else:
        cut_num = round(dist2 / interval) + int(dist2 * 2 < interval)
        short_edge = dist1
    return cut_num, short_edge


base_path = Path(r'D:\UAV_DATA_NEW\dataset_bin\061303')

for i in range(1, 29):
    test_path = base_path / f'test/{i:02d}'
    test_path.mkdir(parents=True, exist_ok=True)
    train_path = base_path / f'train/{i:02d}'
    train_path.mkdir(parents=True, exist_ok=True)


In [8]:
# 二值图像样本生成
src = rasterio.open(r"D:\UAV_DATA_NEW\train_sample\061303.tif")

num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(src.read(1), connectivity=4)

size = 224

for i in range(1, num_labels):
    y0, x0, h, w, num_pixels = stats[i]

    if h > size or w > size:
        continue
    if num_pixels < 32:
        continue

    transform = src.window_transform(Window(y0, x0, h, w))

    xs, ys = np.where(labels[x0:x0 + w, y0:y0 + h] == i)

    output = np.zeros((size, size), dtype=np.uint8)

    output[xs + (size - w) // 2, ys + (size - h) // 2] = 255

    shape_results = shapes(output, transform=transform, mask=output)

    for string, value in shape_results:
        polygon = shape(string)
        cut_num, short_edge = cut_polygon(polygon)

        if short_edge >= 1.2:
            cut_num = round(polygon.area / mean_area)
        if cut_num > 28: 
            continue

        rand = random()

        if rand > 0.8:
            path = base_path / f'test/{cut_num:02d}/{i}.png'
        else:
            path = base_path / f'train/{cut_num:02d}/{i}.png'

        cv2.imwrite(str(path), output)

In [10]:
src = rasterio.open(r"D:\UAV_DATA_NEW\train_sample_dia\061301_dilated.tif")
src_color = rasterio.open(r"D:\UAV_DATA_NEW\train_raw\061301.tif").read([1, 2, 3])

num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(src.read(1), connectivity=4)

size = 224

for i in range(1, num_labels):
    y0, x0, h, w, num_pixels = stats[i]

    if h > size or w > size:
        continue
    if num_pixels < 32:
        continue

    transform = src.window_transform(Window(y0, x0, h, w))

    xs, ys = np.where(labels[x0:x0 + w, y0:y0 + h] == i)

    output = np.zeros((3, size, size), dtype=np.uint8)

    raw_img_area = src_color[:, x0:x0 + w, y0:y0 + h]
    output[:, xs + (size - w) // 2, ys + (size - h) // 2] = raw_img_area[:, xs, ys]

    bin_image = (output[0] > 0).copy().astype(np.uint8)

    shape_results = shapes(bin_image, transform=transform, mask=bin_image)

    for string, value in shape_results:
        polygon = shape(string)
        cut_num, short_edge = cut_polygon(polygon)

        if short_edge >= 1.2:
            cut_num = round(polygon.area / mean_area)
        if cut_num > 28: continue
        rand = random()

        if rand > 0.8:
            path = base_path / f'test/{cut_num:02d}/{i}.png'
        else:
            path = base_path / f'train/{cut_num:02d}/{i}.png'

        cv2.imwrite(str(path), np.transpose(output, (1, 2, 0)))